### Load source code

In [316]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [317]:
import sys
sys.path.append('../')

In [445]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd
import source.players.bayesian_approximator
import source.belief as belief

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import shutil
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [446]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [447]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [572]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)
reload(belief)

<module 'source.belief' from '..\\source\\belief.py'>

### Set batch path, batch folder need not to already exist

In [573]:
experiments_folder = "../Experiments"
exp_subfolder = "/c1" #insert folder where batch is
batch_name = "test_multi" #insert batch name
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Parsing the csv to generate a batch containing the configurations and running them

In [574]:
b = runner.Batch(batch_path, experiments_folder + exp_subfolder)

In [575]:
if os.path.isdir(experiments_folder + exp_subfolder + "/" + batch_name):
    shutil.rmtree(experiments_folder + exp_subfolder + "/" + batch_name)
b.parse_batch()

In [576]:
batches = []
batches.append(b)

In [577]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=3)

### Collecting results of the runs

In [578]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"],
                      "tb": c.stats["theoretical_bound"]}
                     for c in confs]
            util.plot_dicts(dlist, ylabel="$exp\_reg(t)$", xlabel="$t$", save=True, show=False, semilog=False,
                            path=b.results_folder_path,
                            title=confs[0].game.players[0].__class__.name + " vs " + confs[0].game.players[1].__class__.name)

### Checking everything was fine by running the configuration "step-by-step"

In [555]:
cf = deepcopy(batches[0].configurations[0])

In [556]:
gm = cf.game

In [557]:
e = runner.Experiment(gm)

In [558]:
for i in range(50):
    #e.run_multi_profile_interaction()
    #e.run_partial_feedback_interaction()
    e.run_interaction()
    print(e.game.players[1])
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2])

sta1
{0: [0.505949358963137, 0.0, 0.01698733098743571, 0.47706331004942626, 0.0, 0.0], 1: [0, 0, 0.0, 0, 0, 0], 2: [0.1021021021021021, 0.012012012012012014, 0.12312312312312314, 0.15915915915915918, 0.1831831831831832, 0.42042042042042044]}
{1: [2], 2: [5], 0: [5, 0]}
sta1
{0: [0.505949358963137, 0.0, 0.01698733098743571, 0.47706331004942626, 0.0, 0.0], 1: [0, 0, 0.0, 0, 0, 0], 2: [0.1021021021021021, 0.012012012012012014, 0.12312312312312314, 0.15915915915915918, 0.1831831831831832, 0.42042042042042044]}
{1: [2], 2: [4], 0: [5, 0]}
sta1
{0: [0.6575342465753428, 0.0, 0.21818181818181792, 0.6315789473684206, 0.0928143712574847, 0.3998906166169339], 1: [0, 0.0, 0, 0, 0, 0], 2: [0.1021021021021021, 0.012012012012012014, 0.12312312312312314, 0.15915915915915918, 0.1831831831831832, 0.42042042042042044]}
{1: [1], 2: [2], 0: [0, 5]}
sta1
{0: [0.6575342465753428, 0.0, 0.21818181818181792, 0.6315789473684206, 0.0928143712574847, 0.3998906166169339], 1: [0, 0.0, 0, 0, 0, 0], 2: [0.102102102102

sta1
{0: [0.6575342465753428, 0.0, 0.21818181818181792, 0.6315789473684206, 0.0928143712574847, 0.3998906166169339], 1: [0, 0.0, 0, 0, 0, 0], 2: [0.1021021021021021, 0.012012012012012014, 0.12312312312312314, 0.15915915915915918, 0.1831831831831832, 0.42042042042042044]}
{1: [1], 2: [5], 0: [4, 3]}
sta1
{0: [0.6575342465753428, 0.0, 0.21818181818181792, 0.6315789473684206, 0.0928143712574847, 0.3998906166169339], 1: [0, 0.0, 0, 0, 0, 0], 2: [0.1021021021021021, 0.012012012012012014, 0.12312312312312314, 0.15915915915915918, 0.1831831831831832, 0.42042042042042044]}
{1: [1], 2: [4], 0: [0, 5]}
sta1
{0: [0.6575342465753428, 0.0, 0.21818181818181792, 0.6315789473684206, 0.0928143712574847, 0.3998906166169339], 1: [0, 0.0, 0, 0, 0, 0], 2: [0.1021021021021021, 0.012012012012012014, 0.12312312312312314, 0.15915915915915918, 0.1831831831831832, 0.42042042042042044]}
{1: [1], 2: [4], 0: [5, 3]}
sta1
{0: [0.6575342465753428, 0.0, 0.21818181818181792, 0.6315789473684206, 0.0928143712574847, 0.39

In [492]:
print(gm.history)

[{1: [0], 2: [0], 0: [2, 5]}]
